In [1]:
pip install -U pandasql

Requirement already up-to-date: pandasql in c:\users\crie072\anaconda3\lib\site-packages (0.7.3)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

from pandasql import sqldf

In [3]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20201013


毎週共有されるCSCからのセキュポスデータ（新規入会）を取り込み、Datastudio読込用に編集する

In [4]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [5]:
#ディレクトリ指定
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\承認データ")

#input
input_file = "承認データ.xlsx"

#抽出する日付指定
yyyymmdd = 20201006

#out_put
output_file = "承認データdatastudio読込用"

In [6]:
print(yyyymmdd)

20201006


新規Vintage CUSTデータの読込＆集計

In [7]:
df = pd.read_excel(data_dir /f"{input_file}",
                  dtype={'ｶｰﾄﾞ加入日':int},encoding='shift-jis')
df = df[df['ｶｰﾄﾞ加入日'] >= yyyymmdd]

In [8]:
df.head()

,ｶｰﾄﾞ加入日,続柄,ｶｰﾄﾞ提携会社(1),ｶｰﾄﾞ提携会社(2),ｶｰﾄﾞ種類ｺｰﾄﾞ,ﾌﾞﾗﾝﾄﾞｺｰﾄﾞ,切替ｺｰﾄﾞ,勧誘ｺｰﾄﾞ,受付部支店ｺｰﾄﾞ,再発行ｺｰﾄﾞ,ｶｰﾄﾞ種類（旧）,ﾌﾞﾗﾝﾄﾞ（旧）,ﾘﾎﾞ支払方法ｺｰﾄﾞ,勧誘団体コード,カード提携特殊エリア,エンボス内容,ｶｰﾄﾞ年会費ｺｰﾄﾞ
40690,20201006,0,D00,1,2,1,0,3,104,0,NaN,NaN,12.0,4000,NaN,4.000000e+03,0
40691,20201006,0,D00,8,31,1,0,3,103,0,NaN,NaN,52.0,3000,NaN,3.000000e+03,2
40692,20201006,2,D02,1,2,1,0,3,102,0,NaN,NaN,12.0,2000,NaN,2.000010e+09,2
40693,20201006,0,D02,1,2,1,0,3,102,0,NaN,NaN,12.0,2000,NaN,2.000010e+09,2
40694,20201006,0,D02,1,2,1,0,3,174,0,NaN,NaN,12.0,11000,NaN,1.100001e+10,2


In [9]:
df = df.rename(columns={'ｶｰﾄﾞ加入日':'kanyu_date',
                        '続柄':'zokugara',
                        'ｶｰﾄﾞ提携会社(1)':'teikei_1',
                        'ｶｰﾄﾞ提携会社(2)':'teikei_2',
                        '切替ｺｰﾄﾞ':'kirikae',
                        'カード提携特殊エリア':'tokushu',
                        '受付部支店ｺｰﾄﾞ':'uketuke',
                        'ｶｰﾄﾞ種類ｺｰﾄﾞ':'card'})
df.head()

,kanyu_date,zokugara,teikei_1,teikei_2,card,ﾌﾞﾗﾝﾄﾞｺｰﾄﾞ,kirikae,勧誘ｺｰﾄﾞ,uketuke,再発行ｺｰﾄﾞ,ｶｰﾄﾞ種類（旧）,ﾌﾞﾗﾝﾄﾞ（旧）,ﾘﾎﾞ支払方法ｺｰﾄﾞ,勧誘団体コード,tokushu,エンボス内容,ｶｰﾄﾞ年会費ｺｰﾄﾞ
40690,20201006,0,D00,1,2,1,0,3,104,0,NaN,NaN,12.0,4000,NaN,4.000000e+03,0
40691,20201006,0,D00,8,31,1,0,3,103,0,NaN,NaN,52.0,3000,NaN,3.000000e+03,2
40692,20201006,2,D02,1,2,1,0,3,102,0,NaN,NaN,12.0,2000,NaN,2.000010e+09,2
40693,20201006,0,D02,1,2,1,0,3,102,0,NaN,NaN,12.0,2000,NaN,2.000010e+09,2
40694,20201006,0,D02,1,2,1,0,3,174,0,NaN,NaN,12.0,11000,NaN,1.100001e+10,2


In [10]:
q = """
        SELECT
        kanyu_date
        ,case 
            when substr(tokushu,1,2) = "20" then '店頭タブレット'
            when uketuke = "299" then 'オンライン'
            when uketuke in ("200","001") then '郵送'
            else '店頭紙' 
            end as uketsuke_channel
        ,case 
            when kirikae = "0" then 'N/A'
            when substr(tokushu,1,2) = "80" then 'DM'
            when substr(tokushu,1,2) = "81" then "コンタクト"
            when substr(tokushu,1,2) = "86" then "店頭"
            else "店頭"
            end as kirikae_channel
        ,case
            when teikei_2 =50 then '臨時'
            when teikei_1 ="D02" then '外商'
            when teikei_2 ="05" then 'ハウスゴールド'
            when card ="02"  then 'ゴールド'
            else '一般'
            end as card_shubetsu
        ,case 
            when zokugara = 0 then '本人'
            else '家族' 
            end as kazoku_shubetsu
        ,sum(case when kirikae = "0" then 1 else 0 end ) as no_of_new
        ,sum(case when kirikae = "1" then 1 else 0 end ) as no_of_kirikae
        FROM 
          df
        where
          teikei_2 not in ("95") 
        group by
        1,2,3,4,5
        """

In [11]:
df_sql = sqldf(q, locals())
df_sql['kanyu_date']=pd.to_datetime(df_sql['kanyu_date'], format='%Y%m%d')
df_sql['kanyu_year'] = df_sql['kanyu_date'].dt.year
df_sql['kanyu_month'] = df_sql['kanyu_date'].dt.month
df_sql['kanyu_day'] = df_sql['kanyu_date'].dt.day
df_sql['kanyu_date'] = df_sql['kanyu_date'].dt.strftime('%Y/%m/%d')
df_sql.head()

,kanyu_date,uketsuke_channel,kirikae_channel,card_shubetsu,kazoku_shubetsu,no_of_new,no_of_kirikae,kanyu_year,kanyu_month,kanyu_day
0,2020/10/06,オンライン,N/A,ゴールド,本人,1,0,2020,10,6
1,2020/10/06,オンライン,N/A,一般,本人,76,0,2020,10,6
2,2020/10/06,店頭タブレット,N/A,ゴールド,家族,2,0,2020,10,6
3,2020/10/06,店頭タブレット,N/A,ゴールド,本人,8,0,2020,10,6
4,2020/10/06,店頭タブレット,N/A,一般,本人,159,0,2020,10,6


CSVアウトプット

In [12]:
df_sql.to_csv(data_dir /f"{output_file}.csv", index=None,header=True,encoding='utf-8-sig')